# 1) Getting started


You can install required packages and dependencies with either apt-get or pip. 
The cobrapy package will come with *Salmonella* and *E. coli* models, as well as an *E. coli* textbook core metabolism model. We are going to load the textbook model, "ecoli" or "salmonella" would also be valid arguements.



```
# This is formatted as code
```

## 1.1) Install dependencies and load cobrapy

In [0]:
!pip install -q cobra

In [0]:
from __future__ import print_function

import cobra
import cobra.test

model = cobra.test.create_test_model("textbook")

## 1.2) Inspect the model

Now let's have a first glance at the model.



In [0]:
model

The reactions, metabolites, and genes attributes of the cobrapy model are a special type of list called a `cobra.DictList`, and each one is made up of `cobra.Reaction`, `cobra.Metabolite` and `cobra.Gene` objects respectively.

In [0]:
model.reactions

In [0]:
model.metabolites

In [0]:
model.genes

Just like a regular list, objects in the `DictList` can be retrieved by index. For example, to get the 17th reactions in the model (at index 16 because of 0-indexing):

In [0]:
model.reactions[16]

##1.3) Reactions, Metabolites and Genes

### 1.3.1) Reactions

We will consider the reaction glucose 6-phosphate isomerase, which interconverts glucose 6-phosphate and fructose 6-phosphate. The reaction id for this reaction in our test model is PGI.

In [0]:
pgi = model.reactions.get_by_id("PGI")
pgi

We can view the full name and reaction catalyzed as strings

In [0]:
print(pgi.name)
print(pgi.reaction)

We can also view reaction upper and lower bounds. Because the pgi.lower_bound < 0, and pgi.upper_bound > 0, pgi is reversible.

In [0]:
print(pgi.lower_bound, "< pgi <", pgi.upper_bound)
print(pgi.reversibility)

We can also ensure the reaction is mass balanced. This function will return elements which violate mass balance. If it comes back empty, then the reaction is mass balanced.

In [0]:
pgi.check_mass_balance()

In order to add a metabolite, we pass in a `dict` with the metabolite object and its coefficient

In [0]:
pgi.add_metabolites({model.metabolites.get_by_id("h_c"): -1})
pgi.reaction

The reaction is no longer mass balanced

In [0]:
pgi.check_mass_balance()

We can remove the metabolite, and the reaction will be balanced once again.

In [0]:
pgi.subtract_metabolites({model.metabolites.get_by_id("h_c"): -1})
print(pgi.reaction)
print(pgi.check_mass_balance())

It is also possible to build the reaction from a string. However, care must be taken when doing this to ensure reaction id's match those in the model. The direction of the arrow is also used to update the upper and lower bounds.

In [0]:
pgi.reaction = "g6p_c --> f6p_c + h_c + green_eggs + ham"

In [0]:
pgi.reaction

In [0]:
pgi.reaction = "g6p_c <=> f6p_c"
pgi.reaction

###1.3.2) Metabolites

We will consider cytosolic atp as our metabolite, which has the id `"atp_c"` in our test model.

In [0]:
atp = model.metabolites.get_by_id("atp_c")
atp

We can print out the metabolite name and compartment (cytosol in this case) directly as string.

In [0]:
print(atp.name)
print(atp.compartment)

We can see that ATP is a charged molecule in our model.

In [0]:
atp.charge

We can see the chemical formula for the metabolite as well.

In [0]:
print(atp.formula)

The reactions attribute gives a `frozenset` of all reactions using the given metabolite. We can use this to count the number of reactions which use atp.

In [0]:
len(atp.reactions)

A metabolite like glucose 6-phosphate will participate in fewer reactions.

In [0]:
model.metabolites.get_by_id("g6p_c").reactions

###1.3.3) Genes

The `gene_reaction_rule` is a boolean representation of the gene requirements for this reaction to be active as described in [Schellenberger et al 2011 Nature Protocols 6(9):1290-307](http://dx.doi.org/doi:10.1038/nprot.2011.308).

The GPR is stored as the gene_reaction_rule for a Reaction object as a string.

In [0]:
gpr = pgi.gene_reaction_rule
gpr

Corresponding gene objects also exist. These objects are tracked by the reactions itself, as well as by the model

In [0]:
pgi.genes

In [0]:
pgi_gene = model.genes.get_by_id("b4025")
pgi_gene

Each gene keeps track of the reactions it catalyzes

In [0]:
pgi_gene.reactions

Altering the gene_reaction_rule will create new gene objects if necessary and update all relationships.

In [0]:
pgi.gene_reaction_rule = "(spam or eggs)"
pgi.genes

In [0]:
pgi_gene.reactions

Newly created genes are also added to the model

In [0]:
model.genes.get_by_id("spam")

The undelete_model_genes can be used to reset a gene deletion

#2) Building a model

This simple example demonstrates how to create a model, create a reaction, and then add the reaction to the model.

We'll use the '3OAS140' reaction from the STM_1.0 model:

1.0 malACP[c] + 1.0 h[c] + 1.0 ddcaACP[c] $\rightarrow$ 1.0 co2[c] + 1.0 ACP[c] + 1.0 3omrsACP[c]

First, create the model and reaction.

In [0]:
from cobra import Model, Reaction, Metabolite
# Best practise: SBML compliant IDs
model = Model('example_model')

reaction = Reaction('3OAS140')
reaction.name = '3 oxoacyl acyl carrier protein synthase n C140 '
reaction.subsystem = 'Cell Envelope Biosynthesis'
reaction.lower_bound = 0.  # This is the default
reaction.upper_bound = 1000.  # This is the default

We need to create metabolites as well. If we were using an existing model, we could use `Model.get_by_id` to get the appropriate Metabolite objects instead.

In [0]:
ACP_c = Metabolite(
    'ACP_c',
    formula='C11H21N2O7PRS',
    name='acyl-carrier-protein',
    compartment='c')
omrsACP_c = Metabolite(
    '3omrsACP_c',
    formula='C25H45N2O9PRS',
    name='3-Oxotetradecanoyl-acyl-carrier-protein',
    compartment='c')
co2_c = Metabolite('co2_c', formula='CO2', name='CO2', compartment='c')
malACP_c = Metabolite(
    'malACP_c',
    formula='C14H22N2O10PRS',
    name='Malonyl-acyl-carrier-protein',
    compartment='c')
h_c = Metabolite('h_c', formula='H', name='H', compartment='c')
ddcaACP_c = Metabolite(
    'ddcaACP_c',
    formula='C23H43N2O8PRS',
    name='Dodecanoyl-ACP-n-C120ACP',
    compartment='c')

Adding metabolites to a reaction requires using a dictionary of the metabolites and their stoichiometric coefficients. A group of metabolites can be added all at once, or they can be added one at a time.

In [0]:
reaction.add_metabolites({
    malACP_c: -1.0,
    h_c: -1.0,
    ddcaACP_c: -1.0,
    co2_c: 1.0,
    ACP_c: 1.0,
    omrsACP_c: 1.0
})

reaction.reaction  # This gives a string representation of the reaction

The gene_reaction_rule is a boolean representation of the gene requirements for this reaction to be active as described in [Schellenberger et al 2011 Nature Protocols 6(9):1290-307](http://dx.doi.org/doi:10.1038/nprot.2011.308). We will assign the gene reaction rule string, which will automatically create the corresponding gene objects.

In [0]:
reaction.gene_reaction_rule = '( STM2378 or STM1197 )'
reaction.genes

At this point in time, the model is still empty

In [0]:
print('%i reactions initially' % len(model.reactions))
print('%i metabolites initially' % len(model.metabolites))
print('%i genes initially' % len(model.genes))

We will add the reaction to the model, which will also add all associated metabolites and genes

In [0]:
model.add_reactions([reaction])

# Now there are things in the model
print('%i reaction' % len(model.reactions))
print('%i metabolites' % len(model.metabolites))
print('%i genes' % len(model.genes))

We can iterate through the model objects to observe the contents

In [0]:
# Iterate through the the objects in the model
print("Reactions")
print("---------")
for x in model.reactions:
    print("%s : %s" % (x.id, x.reaction))

print("")
print("Metabolites")
print("-----------")
for x in model.metabolites:
    print('%9s : %s' % (x.id, x.formula))

print("")
print("Genes")
print("-----")
for x in model.genes:
    associated_ids = (i.id for i in x.reactions)
    print("%s is associated with reactions: %s" %
          (x.id, "{" + ", ".join(associated_ids) + "}"))

Last we need to set the objective of the model. Here, we just want this to be the maximization of the flux in the single reaction we added and we do this by assigning the reaction's identifier to the `objective` property of the model.

In [0]:
model.objective = '3OAS140'

The created objective is a symbolic algebraic expression and we can examine it by printing it

In [0]:
print(model.objective.expression)
print(model.objective.direction)

which here shows that the solver will maximize the flux in the forward direction.

#3) Simulating

Simulations using flux balance analysis can be solved using `Model.optimize()`. This will maximize or minimize (maximizing is the default) flux through the objective reactions.

##3.1) Running FBA


Get the objective of a model

In [0]:
model = cobra.test.create_test_model("textbook")

In [0]:
print(model.objective)

In [0]:
biomass_rxn = model.reactions.get_by_id('Biomass_Ecoli_core')
biomass_rxn

In [0]:
biomass_rxn.id = 'BIOMASS_Ecoli_core_w_GAM'

In [0]:
solution = model.optimize()
print(solution)

The Model.optimize() function will return a Solution object. A solution object has several attributes:

 - `objective_value`: the objective value
 - `status`: the status from the linear programming solver
 - `fluxes`: a pandas series with flux indexed by reaction identifier. The flux for a reaction variable is the difference of the primal values for the forward and reverse reaction variables. (More about [Pandas Series and Dataframes](https://pandas.pydata.org))
 - `shadow_prices`: a pandas series with shadow price indexed by the metabolite identifier.

For example, after the last call to `model.optimize()`, if the optimization succeeds it's status will be optimal. In case the model is infeasible an error is raised.

In [0]:
solution.objective_value

Get flux distribution of the current solution:

In [0]:
solution.fluxes

###3.1.1) Analyzing FBA solutions

Models solved using FBA can be further analyzed by using summary methods, which output printed text to give a quick representation of model behavior. Calling the summary method on the entire model displays information on the input and output behavior of the model, along with the optimized objective.

In [0]:
model.summary()

In addition, the input-output behavior of individual metabolites can also be inspected using summary methods. For instance, the following commands can be used to examine the overall redox balance of the model

In [0]:
model.metabolites.nadh_c.summary()

Or to get a sense of the main energy production and consumption reactions

In [0]:
model.metabolites.atp_c.summary()

###3.1.2) Visualizing FBA solutions with Escher Maps



Create a directory for results and save flux solution in a json file.

In [0]:
import os
from google.colab import files

directory = "results"
if not os.path.exists(directory):
    os.makedirs(directory)
       
solution.fluxes.to_json('results/flux_1.json')
#files.download('results/flux_1.json')

To view the flux solution on a map:



1.   Go to [Escher Maps](https://escher.github.io)
2.   Choose *Escherichia Coli* as organism, *Core metabolism (e_coli_core)* as map, *e_coli_core* as model and *Viewer* as Tool
3.   Press *Load map*
4.  You will be redirected to the metabolic map of E.coli
5. To map your flux solution, go to the *Menu > Data > Load reaction data* and choose the* json file*, you just downloaded
6. Check the visualisation to your likings, go to *Menu > View > Settings*




###3.1.3) Changing the Objectives

The objective function is determined from the objective_coefficient attribute of the objective reaction(s). Generally, a "biomass" function which describes the composition of metabolites which make up a cell is used.

In [0]:
biomass_rxn

The objective function can be changed by assigning Model.objective, which can be a reaction object (or just it's name), or a `dict` of `{Reaction: objective_coefficient}`.

In [0]:
ATPM_rxn = model.reactions.get_by_id("ATPM")
ATPM_rxn

In [0]:
# change the objective to ATPM
model.objective = "ATPM"

In [0]:
solution = model.optimize()
solution.objective_value

In [0]:
solution.fluxes.to_json('results/flux_2.json')
#files.download('results/flux_2.json')


Go to again to [Escher Maps](https://escher.github.io) and load the flux solution.

So far, we always searched for the maximum:

In [0]:
model.objective_direction

but as for maintenace in terms of ATP minimisation might make more sense. You can simple set it like this:

In [0]:
model.objective_direction = 'min'
model.objective_direction

In [0]:
solution = model.optimize()
solution.objective_value

In [0]:
solution.fluxes.to_json('results/flux_3.json')
#files.download('results/flux_3.json')

Go to again to [Escher Maps](https://escher.github.io) and load the flux solution.

# 4) omitted

# 5) References

This material was compiled using https://cobrapy.readthedocs.io/en/latest/ and is influenced by the deNBI/IPK CompBiol Starter 2019 on Metabolic Modelling (Dr Blätke/Dr Szymanski).

 10.1186/1752-0509-7-74